In [3]:
import os
import copy
import numpy as np
import pickle as pkl
import pandas as pd
from collections import defaultdict
from datetime import datetime
import cv2
def parse_and_fuse_timestamp(time_str, time_nano):
    # Parse the time string (e.g., "2024-04-17 09:19:41") and timeNano (e.g., "322223")
    dt = datetime.strptime(f"{time_str}_{time_nano.split('_')[-1]}", "%Y-%m-%d_%H_%M_%S_%f")
    return dt

def process_pkl_files(meta_dir_root_dir, output_dir, background_image_name):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Dictionary to store tracks, where each key is a Label and value is a list of detections
    tracks = defaultdict(list)

    # create a frame with the background image
    background_image = cv2.imread(background_image_name)
    # Iterate through all .pkl files in the Meta directory
    meta_dir = os.path.join(meta_dir_root_dir, 'Meta')
    list_of_pkl_files = os.listdir(meta_dir)
    image_dir = os.path.join(meta_dir_root_dir, 'Images')
    list_of_jpg_files = os.listdir(image_dir)
    # max_x = 0
    # max_y = 0
    for pkl_indx, pkl_file in enumerate(list_of_pkl_files):
        # background_image_copy = copy.deepcopy(background_image)
        if pkl_file.endswith('.pkl'):
            # frame_bbox_image = cv2.imread(os.path.join(image_dir, list_of_jpg_files[pkl_indx]))
            pkl_path = os.path.join(meta_dir, pkl_file)
            # Read the .pkl file
            with open(pkl_path, 'rb') as f:
                data = pkl.load(f)

            # Extract required fields
            label = data['Label']
            time_nano = data['timeNano']
            time = data['time']
            box = data['BOX']

            # Fuse time and timeNano for sorting
            fused_timestamp = parse_and_fuse_timestamp(time, time_nano)

            # change fused_timestamp to a counter in a resolution of 1/30 seconds from 1/1/1900
            fused_timestamp = int((fused_timestamp - datetime(1975, 1, 1)).total_seconds() * 30)

            # Convert BOX [x1, y1, x2, y2] to four points [x1, y1, x2, y1, x2, y2, x1, y2]
            y1, x1, y2, x2 = [int(x) for x in box]
            box_points = [x1, y1, x2, y1, x1, y2, x2, y2]
            # if max_x < x2:
            #     max_x = x2
            # if max_y < y2:
            #     max_y = y2
            # #put the frame_bbox_image inside backjground_image at location of box_points
            # background_image_copy[x1:x2, y1:y2] = frame_bbox_image
            # cv2.imwrite(os.path.join(output_dir, background_image_name), background_image_copy)

            # print(label)
            # Append detection to the track for this Label
            tracks[label].append({
                'timestamp': fused_timestamp,
                'box_points': box_points,
                'source_file_name':pkl_file.split('.pkl')[0]
                # 'background_image': background_image_copy
            })
            print(f'\rprocessing pkl #{pkl_indx}/{len(list_of_pkl_files)}', end="")

    # print(f'max_y={max_y}')
    # print(f'max_x = {max_x}')
    # Process each track (Label) and save its detections to a CSV file
    indx = 0
    for label, detections in tracks.items():
        indx+=1
        print(f'\rprocessing track #{indx}/{len(list_of_pkl_files)}', end="")

        # Sort detections by timestamp to ensure chronological order within the track
        detections = sorted(detections, key=lambda x: x['timestamp'])

        # Prepare data for CSV: each row is a detection in the track
        csv_data = []
        for i, detection in enumerate(detections, 0):  # 0-based index for detections
            box_points = detection['box_points']
            # Pad box points with zeros to reach 32 values
            padded_points = box_points + [0.0] * (33 - len(box_points))
            # Create row: detection index + box points + final zero
            row = [detection['timestamp']] + padded_points + [0.0] + [detection['source_file_name']]
            csv_data.append(row)

        # Create a DataFrame without headers for the track
        df = pd.DataFrame(csv_data)

        # Define CSV filename (e.g., track_001.csv)
        csv_filename = f"{str(label).zfill(3)}.csv"
        csv_path = os.path.join(output_dir, csv_filename)

        # Save the track's detections to CSV without headers
        df.to_csv(csv_path, index=False, header=False)
        print(f"Saved track for label {label} to {csv_path} with {len(detections)} detections")

def main():
    string_date = '2023_2_9'
    training_or_testing_str = 'training'
    # training_or_testing_str = 'testing'
    # Define paths
    meta_dir = '/home/pp/Desktop/datasets/elsec_dataset/frame/01/' + string_date + '/Pos'  # Path to Meta directory
    output_dir = '/home/pp/Desktop/datasets/trajrec_data/elsec_data/'+ training_or_testing_str+'/trajectories/' + string_date  # Path to save CSV files
    background_image_name = '/home/pp/Desktop/datasets/elsec_dataset/frame/01/elsec_road_meshulash_times_check.jpg'
    # Process the .pkl files and save tracks
    process_pkl_files(meta_dir, output_dir, background_image_name)

if __name__ == '__main__':
    main()

processing pkl #509514/509515max_y=480
max_x = 640
processing track #1/509515Saved track for label 106240 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/training/trajectories/2023_2_9/106240.csv with 33 detections
processing track #2/509515Saved track for label 89992 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/training/trajectories/2023_2_9/89992.csv with 724 detections
processing track #3/509515Saved track for label 114305 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/training/trajectories/2023_2_9/114305.csv with 17 detections
processing track #4/509515Saved track for label 114309 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/training/trajectories/2023_2_9/114309.csv with 42 detections
processing track #5/509515Saved track for label 104496 to /home/pp/Desktop/datasets/trajrec_data/elsec_data/training/trajectories/2023_2_9/104496.csv with 8 detections
processing track #6/509515Saved track for label 103030 to /home/pp/Desktop/datasets/trajrec_data/elsec_dat